<a href="https://colab.research.google.com/github/Yushamsi/Data-Ingestion-Combined-Flights/blob/main/Data-Ingestion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# *Data Ingestion*

In [1]:
!pip install dask[dataframe]  # Installing Dask with dataframe support
!pip install "dask[dataframe]"
!pip install "modin[ray]"  # This will install Modin with Ray as a dependency; you can use "modin[dask]" if you prefer Dask
!pip install psutil
!pip install pyyaml  # For yaml support
!pip install dask-expr

!pip install kaggle


zsh:1: no matches found: dask[dataframe]


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
DEPRECATION: colab 1.13.5 has a non-standard dependency specifier pytz>=2011n. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of colab or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
DEPRECATION: colab 1.13.5 has a non-standard dependency specifier pytz>=2011n. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of colab or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
DEPRECATION: colab 1.13.5 has a non-standard dependency specifi

In [2]:
# from google.colab import files
# files.upload()  # Upload the kaggle.json file

# !mkdir -p ~/.kaggle
# !mv kaggle.json ~/.kaggle/
# !chmod 600 ~/.kaggle/kaggle.json

In [8]:
import pandas as pd
import dask
dask.config.set({'dataframe.query-planning': True})
import dask.dataframe as dd
import modin.pandas as mpd
import os
# {"username":"yusufshamsi","key":"94e1539e29c581b6b37d9f23c17b10fc"}
os.environ["KAGGLE_USERNAME"] = "yusufshamsi"
os.environ["KAGGLE_KEY"] = "94e1539e29c581b6b37d9f23c17b10fc"
import psutil
import time
import yaml
import testutility as util
import zipfile
import kaggle

In [4]:
!kaggle datasets download -d robikscube/flight-delay-dataset-20182022

100%|█████████████████████████████████████▉| 3.73G/3.73G [10:34<00:00, 6.79MB/s]
100%|██████████████████████████████████████| 3.73G/3.73G [10:34<00:00, 6.31MB/s]


In [9]:
zip_path = 'flight-delay-dataset-20182022.zip'  # Path to your zip file
extract_to = 'flight-delay-dataset-20182022'  # Directory where files will be extracted

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

print(f"Files extracted to: {extract_to}")

Files extracted to: flight-delay-dataset-20182022


In [10]:
# File path
file_path = 'flight-delay-dataset-20182022/Combined_Flights_2021.csv'
# file_path = '/Users/Yusuf/Desktop/Combined_Flights_2021.csv'

## Reading the file

In [11]:
# Function to measure CPU and memory usage
def measure_usage(library_name, read_function, file_path):
    # Measure before reading
    start_cpu = psutil.cpu_percent(interval=1)
    start_memory = psutil.virtual_memory().used / (1024 ** 3)  # Convert to GB
    start_time = time.time()

    # Read file
    df = read_function(file_path)

    # Measure after reading
    end_cpu = psutil.cpu_percent(interval=1)
    end_memory = psutil.virtual_memory().used / (1024 ** 3)  # Convert to GB
    end_time = time.time()

    # Calculate differences
    cpu_usage = end_cpu - start_cpu
    memory_usage = end_memory - start_memory
    read_time_seconds = end_time - start_time
    read_time_minutes = read_time_seconds / 60

    # Print results
    print(f"{library_name} read time: {read_time_seconds:.2f} seconds ({read_time_minutes:.2f} minutes)")
    print(f"{library_name} CPU usage change: {cpu_usage:.2f}%")
    print(f"{library_name} Memory usage change: {memory_usage:.2f} GB")

Reading with Pandas

In [12]:
# Measure Pandas
measure_usage("Pandas", pd.read_csv, file_path)

Pandas read time: 18.72 seconds (0.31 minutes)
Pandas CPU usage change: 5.70%
Pandas Memory usage change: 7.54 GB


Reading with Dask


In [13]:
# For Dask since the actual data loading operation (like converting to pandas DataFrame) happens when you perform an action (like compute for Dask), the direct approach works differently.
# For these, typically need to wrap the operation in a function if you're doing more than just reading.
measure_usage("Dask", lambda file: dd.read_csv(file).compute(), file_path)

Dask read time: 13.16 seconds (0.22 minutes)
Dask CPU usage change: 0.70%
Dask Memory usage change: 1.94 GB


Reading with Modin (Pandas on Ray)

In [14]:
# Modin with Ray
os.environ["MODIN_ENGINE"] = "ray"  # Use Ray as the backend
measure_usage("Modin Ray", mpd.read_csv, file_path)


2024-02-25 21:08:22,393	INFO worker.py:1724 -- Started a local Ray instance.


Modin Ray read time: 13.11 seconds (0.22 minutes)
Modin Ray CPU usage change: 2.50%
Modin Ray Memory usage change: 4.74 GB


Reading with Modin (Pandas on Ray)

In [15]:
# Modin with Dask
os.environ["MODIN_ENGINE"] = "dask"  # Use Dask as the backend
measure_usage("Modin Dask", mpd.read_csv, file_path)

(raylet) Spilled 2525 MiB, 21 objects, write throughput 1699 MiB/s. Set RAY_verbose_spill_logs=0 to disable this message.


Modin Dask read time: 8.93 seconds (0.15 minutes)
Modin Dask CPU usage change: 12.90%
Modin Dask Memory usage change: -1.25 GB


# Cleaning Data

In [16]:
def clean_column_names(df):
    df.columns = df.columns.str.strip()
    df.columns = df.columns.str.replace(' ', '_', regex=True)
    df.columns = df.columns.str.replace('[^\\w\\s]', '', regex=True)
    return df

In [17]:
#As Reading with Modin (Pandas on Ray) took the least time we will load the data with that
# Modin with Dask
# os.environ["MODIN_ENGINE"] = "dask"  # Use Dask as the backend
df = mpd.read_csv(file_path)

(raylet) Spilled 5051 MiB, 44 objects, write throughput 2693 MiB/s.


In [18]:
# Apply the function to your DataFrame
df_cleaned = clean_column_names(df)

# Display the cleaned column names
print(df_cleaned.columns, '\n\n')
display(df_cleaned.head(), '\n\n')
display(df_cleaned.info())

Index(['FlightDate', 'Airline', 'Origin', 'Dest', 'Cancelled', 'Diverted',
       'CRSDepTime', 'DepTime', 'DepDelayMinutes', 'DepDelay', 'ArrTime',
       'ArrDelayMinutes', 'AirTime', 'CRSElapsedTime', 'ActualElapsedTime',
       'Distance', 'Year', 'Quarter', 'Month', 'DayofMonth', 'DayOfWeek',
       'Marketing_Airline_Network', 'Operated_or_Branded_Code_Share_Partners',
       'DOT_ID_Marketing_Airline', 'IATA_Code_Marketing_Airline',
       'Flight_Number_Marketing_Airline', 'Operating_Airline',
       'DOT_ID_Operating_Airline', 'IATA_Code_Operating_Airline',
       'Tail_Number', 'Flight_Number_Operating_Airline', 'OriginAirportID',
       'OriginAirportSeqID', 'OriginCityMarketID', 'OriginCityName',
       'OriginState', 'OriginStateFips', 'OriginStateName', 'OriginWac',
       'DestAirportID', 'DestAirportSeqID', 'DestCityMarketID', 'DestCityName',
       'DestState', 'DestStateFips', 'DestStateName', 'DestWac', 'DepDel15',
       'DepartureDelayGroups', 'DepTimeBlk', 'TaxiOu

,FlightDate,Airline,Origin,Dest,Cancelled,Diverted,CRSDepTime,DepTime,DepDelayMinutes,DepDelay,...,WheelsOff,WheelsOn,TaxiIn,CRSArrTime,ArrDelay,ArrDel15,ArrivalDelayGroups,ArrTimeBlk,DistanceGroup,DivAirportLandings
0,2021-03-03,SkyWest Airlines Inc.,SGU,PHX,False,False,724,714.0,0.0,-10.0,...,724.0,813.0,5.0,843,-25.0,0.0,-2.0,0800-0859,2,0.0
1,2021-03-03,SkyWest Airlines Inc.,PHX,SGU,False,False,922,917.0,0.0,-5.0,...,940.0,1028.0,3.0,1040,-9.0,0.0,-1.0,1000-1059,2,0.0
2,2021-03-03,SkyWest Airlines Inc.,MHT,ORD,False,False,1330,1321.0,0.0,-9.0,...,1336.0,1445.0,16.0,1530,-29.0,0.0,-2.0,1500-1559,4,0.0
3,2021-03-03,SkyWest Airlines Inc.,DFW,TRI,False,False,1645,1636.0,0.0,-9.0,...,1703.0,1955.0,7.0,2010,-8.0,0.0,-1.0,2000-2059,4,0.0
4,2021-03-03,SkyWest Airlines Inc.,PHX,BFL,False,False,1844,1838.0,0.0,-6.0,...,1851.0,1900.0,3.0,1925,-22.0,0.0,-2.0,1900-1959,2,0.0


'\n\n'

<class 'modin.pandas.dataframe.DataFrame'>
RangeIndex: 6311871 entries, 0 to 6311870
Data columns (total 61 columns):
 #   Column                                   Dtype  
---  ------                                   -----  
 0   FlightDate                               object 
 1   Airline                                  object 
 2   Origin                                   object 
 3   Dest                                     object 
 4   Cancelled                                bool   
 5   Diverted                                 bool   
 6   CRSDepTime                               int64  
 7   DepTime                                  float64
 8   DepDelayMinutes                          float64
 9   DepDelay                                 float64
 10  ArrTime                                  float64
 11  ArrDelayMinutes                          float64
 12  AirTime                                  float64
 13  CRSElapsedTime                           float64
 14  ActualElapsed

None

# YAML

In [19]:
df_cleaned.shape

(6311871, 61)

In [20]:
%%writefile file.yaml
file_type: csv
dataset_name: Combined_Flights
file_name: Combined_Flights_2021.csv
inbound_delimiter: ","
outbound_delimiter: "|"
skip_leading_rows: 1
number_rows: 6311871
number_columns: 61
columns:
    - FlightDate
    - Airline
    - Origin
    - Dest
    - Cancelled
    - Diverted
    - CRSDepTime
    - DepTime
    - DepDelayMinutes
    - DepDelay
    - ArrTime
    - ArrDelayMinutes
    - AirTime
    - CRSElapsedTime
    - ActualElapsedTime
    - Distance
    - Year
    - Quarter
    - Month
    - DayofMonth
    - DayOfWeek
    - Marketing_Airline_Network
    - Operated_or_Branded_Code_Share_Partners
    - DOT_ID_Marketing_Airline
    - IATA_Code_Marketing_Airline
    - Flight_Number_Marketing_Airline
    - Operating_Airline
    - DOT_ID_Operating_Airline
    - IATA_Code_Operating_Airline
    - Tail_Number
    - Flight_Number_Operating_Airline
    - OriginAirportID
    - OriginAirportSeqID
    - OriginCityMarketID
    - OriginCityName
    - OriginState
    - OriginStateFips
    - OriginStateName
    - OriginWac
    - DestAirportID
    - DestAirportSeqID
    - DestCityMarketID
    - DestCityName
    - DestState
    - DestStateFips
    - DestStateName
    - DestWac
    - DepDel15
    - DepartureDelayGroups
    - DepTimeBlk
    - TaxiOut
    - WheelsOff
    - WheelsOn
    - TaxiIn
    - CRSArrTime
    - ArrDelay
    - ArrDel15
    - ArrivalDelayGroups
    - ArrTimeBlk
    - DistanceGroup
    - DivAirportLandings


Overwriting file.yaml


In [21]:
# Load the YAML schema
config_data = util.read_config_file("file.yaml")

# Access file settings and column names from the schema
read_separator = config_data['inbound_delimiter']
write_separator = config_data['outbound_delimiter']
columns = config_data['columns']

print(f"Read Separator: {read_separator}")
print(f"Write Separator: {write_separator}")
print(f"Columns: {columns}")


Read Separator: ,
Write Separator: |
Columns: ['FlightDate', 'Airline', 'Origin', 'Dest', 'Cancelled', 'Diverted', 'CRSDepTime', 'DepTime', 'DepDelayMinutes', 'DepDelay', 'ArrTime', 'ArrDelayMinutes', 'AirTime', 'CRSElapsedTime', 'ActualElapsedTime', 'Distance', 'Year', 'Quarter', 'Month', 'DayofMonth', 'DayOfWeek', 'Marketing_Airline_Network', 'Operated_or_Branded_Code_Share_Partners', 'DOT_ID_Marketing_Airline', 'IATA_Code_Marketing_Airline', 'Flight_Number_Marketing_Airline', 'Operating_Airline', 'DOT_ID_Operating_Airline', 'IATA_Code_Operating_Airline', 'Tail_Number', 'Flight_Number_Operating_Airline', 'OriginAirportID', 'OriginAirportSeqID', 'OriginCityMarketID', 'OriginCityName', 'OriginState', 'OriginStateFips', 'OriginStateName', 'OriginWac', 'DestAirportID', 'DestAirportSeqID', 'DestCityMarketID', 'DestCityName', 'DestState', 'DestStateFips', 'DestStateName', 'DestWac', 'DepDel15', 'DepartureDelayGroups', 'DepTimeBlk', 'TaxiOut', 'WheelsOff', 'WheelsOn', 'TaxiIn', 'CRSArrTime'

# Comparing Schema File to DF

In [22]:
util.col_header_val(df_cleaned, config_data)

column name and column length validation passed


1

In [23]:
print("Columns of DF are:" , df_cleaned.columns)
print("Columns of YAML are:" , config_data['columns'])

Columns of DF are: Index(['flightdate', 'airline', 'origin', 'dest', 'cancelled', 'diverted',
       'crsdeptime', 'deptime', 'depdelayminutes', 'depdelay', 'arrtime',
       'arrdelayminutes', 'airtime', 'crselapsedtime', 'actualelapsedtime',
       'distance', 'year', 'quarter', 'month', 'dayofmonth', 'dayofweek',
       'marketing_airline_network', 'operated_or_branded_code_share_partners',
       'dot_id_marketing_airline', 'iata_code_marketing_airline',
       'flight_number_marketing_airline', 'operating_airline',
       'dot_id_operating_airline', 'iata_code_operating_airline',
       'tail_number', 'flight_number_operating_airline', 'originairportid',
       'originairportseqid', 'origincitymarketid', 'origincityname',
       'originstate', 'originstatefips', 'originstatename', 'originwac',
       'destairportid', 'destairportseqid', 'destcitymarketid', 'destcityname',
       'deststate', 'deststatefips', 'deststatename', 'destwac', 'depdel15',
       'departuredelaygroups', 'd

In [24]:
if util.col_header_val(df, config_data) == 0:
    print("validation failed")
    # write code to reject the file
else:
    print("col validation passed")
    # write the code to perform further action
    # in the pipleine

column name and column length validation passed
col validation passed


# Write DataFrame to Pipe-Separated GZ File

In [25]:
# Define the output file path with .gz extension to indicate gzip compression
output_file_path = 'Cleaned_Combined_Flights_2021.csv.gz'

# Write the DataFrame to a gzipped CSV with pipe separator
df_cleaned.to_csv(output_file_path, sep='|', index=False, compression='gzip')


Please refer to https://modin.readthedocs.io/en/stable/supported_apis/defaulting_to_pandas.html for explanation.
(raylet) Spilled 8714 MiB, 84 objects, write throughput 2611 MiB/s.


In [32]:
file_path = 'Cleaned_Combined_Flights_2021.csv.gz'

# Read the CSV file into a DataFrame
try:
    df = pd.read_csv(file_path, compression='gzip', on_bad_lines='skip')
except Exception as e:
    print(e)

# Calculate total number of rows and columns
total_rows = len(df)
total_columns = len(df.columns)

# Get the file size in megabytes
file_size_bytes = os.path.getsize(file_path)
file_size_mb = file_size_bytes / (1024 * 1024)  # Convert bytes to megabytes

# Print the summary
print(f"Total number of rows: {total_rows}")
print(f"Total number of columns: {total_columns}")
print(f"File size: {file_size_mb:.2f} MB")

Total number of rows: 6175062
Total number of columns: 1
File size: 301.33 MB
